In [6]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [19]:
#Write a function to do the same that I did with the distance_df data set
def process_distance_data(input_file, output_file):
    """
    Load a CSV file, rename a column for easier merging, and save the processed data as a new CSV file.
    
    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the output CSV file.
    """
    distance_df = pd.read_csv(input_file, encoding='latin1')
    distance_df.rename(columns={'country':'mission_country'}, inplace=True)
    distance_df.to_csv(output_file, index=False)

#Apply the function to "C:\Users\HP\Desktop\Felipe\Moises\Distance\Mission_coordinatesok.csv" and save the output to "C:\Users\HP\Desktop\Felipe\Moises\Distance\epp\distance_to_missions_epp.csv"
process_distance_data("C:/Users/HP/Desktop/Felipe/Moises/Distance/Mission_coordinatesok.csv", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")

In [20]:
# Define the function to process shapefiles
def shape_to_csv(department_path):
    # Read and reproject the shp files
    shape = gpd.read_file(department_path)
    shape = shape.to_crs(epsg=4326)
    # Calculate the centroid and extract the longitude and latitude
    shape['centroid'] = shape.centroid
    shape['longitude'] = shape['centroid'].x
    shape['latitude'] = shape['centroid'].y
    # Save as CSV in the same folder as the shapefile for easier access
    output_csv = department_path.replace(".shp", ".csv")
    shape.to_csv(output_csv, index=False)

# Root directory containing the folders and shapefiles
root_dir = "C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files"

# Go through all subdirectories and apply the function
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".shp") and file.startswith("Distritos_"):
            department_path = os.path.join(root, file) 
            shape_to_csv(department_path)


In [21]:
def append_csvs_to_one(output_file, root_dir):
    # List to store individual DataFrames
    all_data = []

    # Walk through all subdirectories to find CSV files
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".csv"):  # Process only CSV files
                file_path = os.path.join(root, file)
                print(f"Appending: {file_path}")
                # Read the CSV and append to the list
                df = pd.read_csv(file_path)
                all_data.append(df)

    # Concatenate all DataFrames into one
    deparmentos_shape_ok = pd.concat(all_data, ignore_index=True)
    # Delete duplicates in CLAVE variable
    deparmentos_shape_ok = deparmentos_shape_ok.drop_duplicates(subset=['CLAVE'])

    # Save the combined DataFrame as a single CSV
    deparmentos_shape_ok.to_csv(output_file, index=False)
    print(f"All CSV files appended and saved to: {output_file}")

#Apply the function
root_dir = "C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files"
output_file = "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv"

append_csvs_to_one(output_file, root_dir)

Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira_shp_data.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\alto_parana\Distritos_Alto_Parana.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\caazapa\Distritos_Caazapa.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira\Distritos_Guaira.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\itapua\Distritos_Itapua.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\misiones\Distritos_Misiones.csv
All CSV files appended and saved to: C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv


In [22]:
from geopy.distance import geodesic

# Function to compute the closest mission for each department
def find_closest_location(distritos, missions):
    # Normalize longitude and latitude, except for rows where DPTO_DESC == 'GUAIRÁ'
    

    results = []

    for index, dep in distritos.iterrows():
        # Skip if department coordinates are invalid
        if pd.isna(dep['latitude']) or pd.isna(dep['longitude']):
            continue

        dep_coords = (dep['latitude'], dep['longitude'])
        
        # Calculate distances to all missions
        missions['distance'] = missions.apply(
            lambda x: geodesic(dep_coords, (x['latitude'], x['longitude'])).kilometers, axis=1
        )
        
        # Find the closest mission
        closest_mission = missions.loc[missions['distance'].idxmin()]
        results.append({
            'department_index': index,
            'department_name': dep.get('DPTO_DESC', 'Unknown'),
            'mission_name': closest_mission['mission_name'],
            'mission_latitude': closest_mission['latitude'],
            'mission_longitude': closest_mission['longitude'],
            'distance_to_mission_km': closest_mission['distance']
        })
    
    return pd.DataFrame(results)

# Example usage
distritos = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv")
missions = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")

# Compute the closest missions
closest_missions_df = find_closest_location(distritos, missions)

# Save the results
closest_missions_df.to_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/closest_missions.csv", index=False)

print("Closest missions have been calculated and saved!")


In [ ]:

#Now, with the distances per district, merge the closest missions with the survey data
# Load the survey data which is in .dta
survey_data = pd.read_stata("C:/Users/HP/Desktop/Felipe/Encuesta/survey_final.dta")
